In [3]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
num_classes = 10
input_shape = (32, 32, 3)

# Load the train and test data splits
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Display shapes of train and test datasets
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [5]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)

I0000 00:00:1762093573.470149   13419 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8786 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:08:00.0, compute capability: 7.5


In [6]:
def create_encoder():
    resnet = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model


encoder = create_encoder()
encoder.summary()

learning_rate = 0.001
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 50
dropout_rate = 0.5
temperature = 0.05

Model: "cifar10-encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 32, 32, 3)      │             7 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50v2 (Functional)         │ (None, 2048)           │    23,564,800 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,564,807 (89.89 MB)

 Trainable params: 23,519,360 (89.72 MB)

 Non-trainable params: 45,447 (177.53 KB)

In [7]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [8]:
encoder = create_encoder()
classifier = create_classifier(encoder)
classifier.summary()

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Model: "cifar10-classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cifar10-encoder (Functional)    │ (None, 2048)           │    23,564,807 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,619,025 (93.91 MB)

 Trainable params: 24,573,578 (93.74 MB)

 Non-trainable params: 45,447 (177.53 KB)

Epoch 1/50


2025-11-02 15:26:57.418467: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


189/189 ━━━━━━━━━━━━━━━━━━━━ 34s 93ms/step - loss: 1.8611 - sparse_categorical_accuracy: 0.3289
Epoch 2/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 18s 97ms/step - loss: 1.4213 - sparse_categorical_accuracy: 0.4922
Epoch 3/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 19s 102ms/step - loss: 1.2511 - sparse_categorical_accuracy: 0.5614
Epoch 4/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 103ms/step - loss: 1.1236 - sparse_categorical_accuracy: 0.6106
Epoch 5/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 19s 101ms/step - loss: 1.0006 - sparse_categorical_accuracy: 0.6534
Epoch 6/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 19s 100ms/step - loss: 1.0245 - sparse_categorical_accuracy: 0.6455
Epoch 7/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 19s 101ms/step - loss: 1.0002 - sparse_categorical_accuracy: 0.6544
Epoch 8/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 19s 101ms/step - loss: 0.8443 - sparse_categorical_accuracy: 0.7130
Epoch 9/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 19s 102ms/step - loss: 0.7753 - sparse_categorical_accuracy: 0.7357
Epoch 10/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 

In [9]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super().__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="cifar-encoder_with_projection-head"
    )
    return model

In [10]:
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs
)

Model: "cifar-encoder_with_projection-head"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cifar10-encoder (Functional)    │ (None, 2048)           │    23,564,807 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       262,272 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,827,079 (90.89 MB)

 Trainable params: 23,781,632 (90.72 MB)

 Non-trainable params: 45,447 (177.53 KB)

Epoch 1/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 29s 89ms/step - loss: 5.3367
Epoch 2/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 103ms/step - loss: 5.0696
Epoch 3/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - loss: 4.9117
Epoch 4/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 21s 108ms/step - loss: 4.7803
Epoch 5/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 106ms/step - loss: 4.6661
Epoch 6/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - loss: 4.5846
Epoch 7/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - loss: 4.5024
Epoch 8/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 106ms/step - loss: 4.4443
Epoch 9/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - loss: 4.3660
Epoch 10/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 106ms/step - loss: 4.3175
Epoch 11/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - loss: 4.2635
Epoch 12/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - loss: 4.2137
Epoch 13/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - loss: 4.1617
Epoch 14/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - loss: 4.1275
Epoch 15/50
189/

In [11]:
classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Epoch 1/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - loss: 0.1321 - sparse_categorical_accuracy: 0.9673
Epoch 2/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0979 - sparse_categorical_accuracy: 0.9735
Epoch 3/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.0977 - sparse_categorical_accuracy: 0.9732
Epoch 4/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.0951 - sparse_categorical_accuracy: 0.9738
Epoch 5/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.0968 - sparse_categorical_accuracy: 0.9734
Epoch 6/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0953 - sparse_categorical_accuracy: 0.9736
Epoch 7/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0940 - sparse_categorical_accuracy: 0.9736
Epoch 8/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0940 - sparse_categorical_accuracy: 0.9744
Epoch 9/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0904 - sparse_categorical_accuracy: 0.9746
Epoch 10/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 25